<a href="https://colab.research.google.com/github/aidanvu1992/DS-Unit-2-Kaggle-Challenge/blob/master/Anh_Vu_DS_Sprint_Challenge_6_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science, Unit 2_
 
# Sprint Challenge: Predict Steph Curry's shots 🏀

For your Sprint Challenge, you'll use a dataset with all Steph Curry's NBA field goal attempts. (Regular season and playoff games, from October 28, 2009, through June 5, 2019.) 

You'll predict whether each shot was made, using information about the shot and the game. This is hard to predict! Try to get above 60% accuracy. The dataset was collected with the [nba_api](https://github.com/swar/nba_api) Python library.

In [0]:
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

In [0]:
# Read data
import pandas as pd
url = 'https://drive.google.com/uc?export=download&id=1fL7KPyxgGYfQDsuJoBWHIWwCAf-HTFpX'
df = pd.read_csv(url)

# Check data shape
assert df.shape == (13958, 20)

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.

To earn a score of "3", also do all the stretch goals.

You are permitted and encouraged to do as much data exploration as you want.

**1. Begin with baselines for classification.** Your target to predict is `shot_made_flag`. What is your baseline accuracy, if you guessed the majority class for every prediction?

**2. Hold out your test set.** Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

**3. Engineer new feature.** Engineer at least **1** new feature, from this list, or your own idea.
- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
- **Opponent**: Who is the other team playing the Golden State Warriors?
- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
- **Made previous shot**: Was Steph Curry's previous shot successful?

**4. Decide how to validate** your model. Choose one of the following options. Any of these options are good. You are not graded on which you choose.
- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
- **Train/validate/test split: random 80/20%** train/validate split.
- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

**5.** Use a scikit-learn **pipeline** to **encode categoricals** and fit a **Decision Tree** or **Random Forest** model.

**6.** Get your model's **validation accuracy.** (Multiple times if you try multiple iterations.) 

**7.** Get your model's **test accuracy.** (One time, at the end.)


**8.** Given a **confusion matrix** for a hypothetical binary classification model, **calculate accuracy, precision, and recall.**

### Stretch Goals
- Engineer 4+ new features total, either from the list above, or your own ideas.
- Make 2+ visualizations to explore relationships between features and target.
- Optimize 3+ hyperparameters by trying 10+ "candidates" (possible combinations of hyperparameters). You can use `RandomizedSearchCV` or do it manually.
- Get and plot your model's feature importances.



In [0]:
# Function and Library
import pandas as pd
import numpy as np
import seaborn as sns
import pandas_profiling

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform

import category_encoders as ce
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform

## 1. Begin with baselines for classification. 

>Your target to predict is `shot_made_flag`. What would your baseline accuracy be, if you guessed the majority class for every prediction?

In [6]:
# Create majority class
baseline = df['shot_made_flag'].value_counts(normalize = True)[0]
print('Baseline accuracy is', baseline)

Baseline accuracy is 0.5270812437311936


## 2. Hold out your test set.

>Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

In [7]:
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,scoremargin_before_shot
0,20900015,4,Stephen Curry,1,11,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,99,249,0,2009-10-28,GSW,HOU,Regular Season,2.0
1,20900015,17,Stephen Curry,1,9,31,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-122,145,1,2009-10-28,GSW,HOU,Regular Season,0.0
2,20900015,53,Stephen Curry,1,6,2,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,-60,129,0,2009-10-28,GSW,HOU,Regular Season,-4.0
3,20900015,141,Stephen Curry,2,9,49,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-172,82,0,2009-10-28,GSW,HOU,Regular Season,-4.0
4,20900015,249,Stephen Curry,2,2,19,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-68,148,0,2009-10-28,GSW,HOU,Regular Season,0.0


In [21]:
df['vtm'].value_counts()

GSW    7087
HOU     471
POR     398
LAC     398
OKC     397
CLE     364
SAS     343
MEM     316
PHX     301
DEN     298
MIN     283
LAL     280
NOP     265
UTA     262
SAC     225
DAL     222
TOR     187
WAS     168
NYK     168
CHI     165
PHI     157
DET     152
BOS     136
CHA     132
MIA     128
IND     125
ORL     121
MIL     117
BKN     110
ATL      93
NOH      58
NJN      31
Name: vtm, dtype: int64

In [10]:
df['game_date'].value_counts()

2013-05-06    35
2019-02-28    33
2016-05-09    32
2019-01-13    32
2013-12-09    32
              ..
2009-12-28     4
2019-04-09     4
2012-03-11     3
2010-12-08     2
2018-03-08     1
Name: game_date, Length: 801, dtype: int64

In [0]:
#Test/train split
df['game_date'] = pd.to_datetime(df['game_date'], infer_datetime_format=True)
condition = (df['game_date'] >= '2018-10-16') & (df['game_date'] <= '2019-06-13')
train = df[~condition]
test = df[condition]

In [12]:
#Check for shape
train.shape, test.shape

((12249, 20), (1709, 20))

## 3. Engineer new feature.

>Engineer at least **1** new feature, from this list, or your own idea.
>
>- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
>- **Opponent**: Who is the other team playing the Golden State Warriors?
>- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
>- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
>- **Made previous shot**: Was Steph Curry's previous shot successful?

    

In [0]:
# Engineer features
def eng_feats(X):

  #Create a df copy
  X = X.copy()

  #Homecourt Advantage
  X['homecourt'] = X['htm'] == 'GSW'

  #Seconds remaining in the period
  X['seconds_remain_period'] = (X['minutes_remaining'] * 60) + X['seconds_remaining']

  #Seconds remaining in the game
  X['seconds_remain_game'] = X['seconds_remain_period'] + 720*(X['period']-1)

  # Extract components from date_recorded, then drop the original column
  X['year'] = X['game_date'].dt.year
  X['month'] = X['game_date'].dt.month
  X['day'] = X['game_date'].dt.day
  X = X.drop(columns='game_date')

  #Drop Constant features
  X = X.drop('player_name', axis = 1)

  return X


In [0]:
train = eng_feats(train)
test = eng_feats(test)

In [0]:
#Arrange data into X features matrix and y target vector
target = 'shot_made_flag'
features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]
X_val = val.drop(columns = target)
y_val = val[target]
X_test = test[features]
y_test = test[target]

## **4. Decide how to validate** your model. 

>Choose one of the following options. Any of these options are good. You are not graded on which you choose.
>
>- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
>- **Train/validate/test split: random 80/20%** train/validate split.
>- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

In [17]:
#Train/validate/test split: random 80/20%
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, 
                              train_size=0.8, test_size = 0.2,
                              shuffle = True,
                              stratify = train['shot_made_flag'],
                              random_state = 42)
train.shape, val.shape

((9799, 24), (2450, 24))

## 5. Use a scikit-learn pipeline to encode categoricals and fit a Decision Tree or Random Forest model.

In [0]:
# Create pipeline
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=108, random_state=42, n_jobs=-1, max_depth=10)
)

#Fit on train
pipeline.fit(X_train, y_train);

## 6.Get your model's validation accuracy

> (Multiple times if you try multiple iterations.)

In [61]:
#Getting train accuracy score
train_pred = pipeline.predict(X_train)
print("Train: ", accuracy_score(y_train, train_pred))

Train:  0.7700785794468823


In [56]:
param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1), 
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.0min finished


In [62]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__max_features': 0.23368688901777368, 'randomforestclassifier__n_estimators': 108, 'simpleimputer__strategy': 'median'}
Cross-validation MAE 0.3497295642412491


In [63]:
# Calculate validation accuracy
print('Validation Accuracy', pipeline.score(X_val, y_val))

Validation Accuracy 0.6522448979591837


## 7. Get your model's test accuracy

> (One time, at the end.)

In [64]:
# Calculate test accuracy
test_pred = pipeline.predict(X_test)
print("Final test accuracy score: ", accuracy_score(y_test, test_pred))

Final test accuracy score:  0.6278525453481568


## 8. Given a confusion matrix, calculate accuracy, precision, and recall.

Imagine this is the confusion matrix for a binary classification model. Use the confusion matrix to calculate the model's accuracy, precision, and recall.

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

### Calculate accuracy 

In [65]:
# Accuracy
TP_TN = 85 + 36
All = 85 + 36 + 58 + 8
Accuracy = TP_TN / All
print(Accuracy)

0.6470588235294118


### Calculate precision

In [0]:
TP = 85

In [72]:
# PRecision
FP = 58
Precision = TP/(TP + FP)
print(Precision)

0.5944055944055944


### Calculate recall

In [71]:
# Recall
FN = 8
Recall = TP/(TP + FN)
print(Recall)

0.9139784946236559
